<a href="https://colab.research.google.com/github/matthewtech-o/Hamoye-Data-Science-Internship/blob/master/Hamoye_DS_Stage_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download nikitarom/planets-dataset/code

planets-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip planets-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: test-jpg-additional/test-jpg-additional/file_5499.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_55.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_550.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5500.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5501.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5502.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5503.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5504.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5505.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5506.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5507.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5508.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5509.jpg  
  inflating: test-jpg-additional/test-jpg-additional/fi

In [7]:
#importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow import keras

# Load data
train = pd.read_csv('/content/planet/planet/train_classes.csv')
test = pd.read_csv('/content/planet/planet/sample_submission.csv')
train_path = '../content/planet/planet/train-jpg/'
test_path = '../content/planet/planet/test-jpg/'
test_additional = '../content/test-jpg-additional/'

In [8]:
train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [9]:
test.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [10]:
import cv2
from tqdm import tqdm

train_images = []
train_labels = []

#read train images
labels = train['tags'].str.get_dummies(sep=' ').columns

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(train.values, miniters=1000):
    img = cv2.imread('../content/planet/planet/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    train_images.append(cv2.resize(img, (32, 32)))
    train_labels.append(targets)

#scale pixel values
train_images = np.array(train_images, np.float16) / 255.
train_labels = np.array(train_labels, np.uint8)

print("Training data: {}, {}".format(train_images.shape, train_labels.shape))

100%|██████████| 40479/40479 [00:55<00:00, 731.14it/s]


Training data: (40479, 32, 32, 3), (40479, 17)


In [11]:
#get number of unique classes in the train set
train['tags'].nunique()

449

In [12]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
/content/planet/planet/test-jpg/test_38023.jpg
/content/planet/planet/test-jpg/test_9391.jpg
/content/planet/planet/test-jpg/test_6355.jpg
/content/planet/planet/test-jpg/test_9309.jpg
/content/planet/planet/test-jpg/test_9311.jpg
/content/planet/planet/test-jpg/test_34627.jpg
/content/planet/planet/test-jpg/test_5899.jpg
/content/planet/planet/test-jpg/test_28454.jpg
/content/planet/planet/test-jpg/test_35166.jpg
/content/planet/planet/test-jpg/test_35068.jpg
/content/planet/planet/test-jpg/test_32023.jpg
/content/planet/planet/test-jpg/test_32284.jpg
/content/planet/planet/test-jpg/test_25623.jpg
/content/planet/planet/test-jpg/test_35973.jpg
/content/planet/planet/test-jpg/test_11138.jpg
/content/planet/planet/test-jpg/test_30460.jpg
/content/planet/planet/test-jpg/test_10153.jpg
/content/planet/planet/test-jpg/test_22483.jpg
/content/planet/planet/test-jpg/test_12766.jpg
/content/planet/planet/test-jpg/test_8922.jpg
/content/planet

In [13]:
# merge test-jpg and test-jpg-additional 
test_images = []

test_jpg = '../content/planet/planet/test-jpg'
test_image_names = os.listdir(test_jpg)

n_test = len(test_image_names)
test_class = test.iloc[:n_test, :]
add_class = test.iloc[n_test:, :]

test_jpg_add = '../content/test-jpg-additional/test-jpg-additional'
test_add_image_names = os.listdir(test_jpg_add)

#read test images
test = pd.read_csv('../content/planet/planet/sample_submission.csv')

for f, tags in tqdm(test_class.values, miniters=1000):
    img = cv2.imread('../content/planet/planet/test-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    test_images.append(cv2.resize(img, (32, 32)))
    
for f, tags in tqdm(add_class.values, miniters=1000):
    img = cv2.imread('../content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    test_images.append(cv2.resize(img, (32, 32)))
    
test_images = np.array(test_images, np.float16) / 255.

100%|██████████| 20522/20522 [00:27<00:00, 733.33it/s]


In [14]:
#Split training data to training and validation sets

x_train = train_images[0:35000]
x_val = train_images[35000:]
y_train = train_labels[0:35000]
y_val = train_labels[35000:]

print("x_train: {}".format(x_train.shape)) 
print("x_val: {}".format(x_val.shape)) 
print("y_train: {}".format(y_train.shape)) 
print("y_val: {}".format(y_val.shape))

x_train: (35000, 32, 32, 3)
x_val: (5479, 32, 32, 3)
y_train: (35000, 17)
y_val: (5479, 17)


In [15]:
#optimization for training deep neural network
#Building a Sequential Feed Forward Network in Keras

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

#Build the model object
model = Sequential()

# Build the input and the hidden layers
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [16]:
#compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
#fit the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128, epochs=3, verbose=1)

# test_loss, test_accuracy = model.evaluate(test_images, labels)
# print('Test loss: {}'.format(test_loss))
# print('Test accuracy: {}'.format(test_accuracy))

Epoch 1/3
274/274 [==============================] - 107s 388ms/step - loss: 0.2521 - accuracy: 0.0292 - val_loss: 0.2028 - val_accuracy: 0.0102
Epoch 2/3
274/274 [==============================] - 106s 386ms/step - loss: 0.2053 - accuracy: 0.0320 - val_loss: 0.1872 - val_accuracy: 0.0283
Epoch 3/3
274/274 [==============================] - 106s 387ms/step - loss: 0.1933 - accuracy: 0.0400 - val_loss: 0.1776 - val_accuracy: 0.0354


In [18]:
from sklearn.metrics import fbeta_score

p_train = model.predict(train_images, batch_size=128,verbose=2)
p_test = model.predict(test_images, batch_size=128,verbose=2)
# print(fbeta_score(y_val, np.array(p_val) > 0.2, beta=2, average='samples'))

317/317 - 26s
479/479 - 39s


In [19]:
# Saving predicted probability and ground truth for Train Dataset
# Compute the best threshold externally

print(labels)
output = pd.DataFrame()
for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]:
    output['class %d' % index] = p_train[:,index-1]
output.to_csv('predicted_probability.csv', index=False)
for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]:
    output['class %d' % index] = train_labels[:,index-1]
output.to_csv('true_label.csv', index=False)

Index(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down',
       'clear', 'cloudy', 'conventional_mine', 'cultivation', 'habitation',
       'haze', 'partly_cloudy', 'primary', 'road', 'selective_logging',
       'slash_burn', 'water'],
      dtype='object')


In [20]:
values_test = (p_test > .222222)*1.0        
values_test = np.array(values_test, np.uint8)

print(values_test)

# Build Submission

test_labels = []
for row in range(values_test.shape[0]):
    test_labels.append(' '.join(labels[values_test[row,:]==1]))
    
submission = test.copy()
submission.drop('tags', axis = 1)
submission['tags'] = test_labels
submission.to_csv('first_sub.csv', index = False)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]]
